In [14]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Available Device is: ", device)

Available Device is:  cpu


In [15]:
data = pd.read_json("./DATA/text/Sarcasm_Headlines_Dataset.json",lines=True)
data.dropna(inplace=True)
print(data.shape)
data.head()

(26709, 3)


,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data["headline"]), np.array(data["is_sarcastic"]), test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

In [18]:
class dataset(Dataset):
  def __init__(self, X, Y):
    self.X = [tokenizer(x,
                        max_length = 100,
                        truncation = True,
                        padding = 'max_length',
                        return_tensors='pt').to(device)
              for x in X
              ]

    self.Y = torch.tensor(Y, dtype = torch.float32).to(device)
  def __len__(self):
    return len(self.X)
  def __getitem__(self, index):
    return self.X[index], self.Y[index]

training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

In [19]:
BATCH_SIZE = 32
EPOCHS = 25
LR = 1e-4

In [20]:
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle= True)
validation_dataloader = DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle= True)
testing_dataloader = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle= True)

In [21]:
class MyModel(nn.Module):
  def __init__(self,bert):
    super(MyModel, self).__init__()
    self.bert = bert
    self.dropout = nn.Dropout(0.25)
    self.linear1 = nn.Linear(768, 384)
    self.linear2 = nn.Linear(384,1)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, input_ids, attention_mask):
    pooled_output = self.bert(input_ids, attention_mask, return_dict = False)[0][:,0]
    output = self.linear1(pooled_output)
    output = self.dropout(output)
    output = self.linear2(output)
    output = self.sigmoid(output)
    return output

In [22]:
for param in bert_model.parameters():
  param.requires_grad = False
model = MyModel(bert_model).to(device)

In [23]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= LR)

In [ ]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

for epoch in range(EPOCHS):
  total_acc_train = 0
  total_loss_train = 0
  total_acc_val = 0
  total_loss_val = 0
  ## Training and Validation
  for indx, data in enumerate(train_dataloader):
    input, label = data

    input.to(device)
    label.to(device)

    prediction = model(input['input_ids'].squeeze(1),
                        input['attention_mask'].squeeze(1)).squeeze(1)


    batch_loss = criterion(prediction, label)

    total_loss_train += batch_loss.item()

    acc = ((prediction).round() == label).sum().item()
    total_acc_train += acc

    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  ## Validation
  with torch.no_grad():
    for indx, data in enumerate(validation_dataloader):
      input, label = data
      input.to(device)
      label.to(device)

      prediction = model(input['input_ids'].squeeze(1),
                      input['attention_mask'].squeeze(1)).squeeze(1)

      batch_loss_val = criterion(prediction, label)
      total_loss_val += batch_loss_val.item()


      acc = ((prediction).round() == label).sum().item()

      total_acc_val += acc


  total_loss_train_plot.append(round(total_loss_train/1000, 4))
  total_loss_validation_plot.append(round(total_loss_val/100, 4))
  total_acc_train_plot.append(round(total_acc_train/(training_data.__len__())*100, 4))
  total_acc_validation_plot.append(round(total_acc_val/(validation_data.__len__())*100, 4))

  print(f'''Epoch no. {epoch + 1} Train Loss: {total_loss_train/1000:.4f} Train Accuracy: {(total_acc_train/(training_data.__len__())*100):.4f} Validation Loss: {total_loss_val/100:.4f} Validation Accuracy: {(total_acc_val/(validation_data.__len__())*100):.4f}''')
  print("="*50)

Epoch no. 1 Train Loss: 0.2596 Train Accuracy: 80.3380 Validation Loss: 0.4826 Validation Accuracy: 83.4498
Epoch no. 2 Train Loss: 0.2120 Train Accuracy: 84.5528 Validation Loss: 0.4491 Validation Accuracy: 85.0974
Epoch no. 3 Train Loss: 0.1996 Train Accuracy: 85.4140 Validation Loss: 0.4221 Validation Accuracy: 86.1208
Epoch no. 4 Train Loss: 0.1920 Train Accuracy: 85.7189 Validation Loss: 0.4137 Validation Accuracy: 86.2956
Epoch no. 5 Train Loss: 0.1868 Train Accuracy: 86.3393 Validation Loss: 0.4111 Validation Accuracy: 86.4204
Epoch no. 6 Train Loss: 0.1856 Train Accuracy: 86.5854 Validation Loss: 0.4034 Validation Accuracy: 86.5701
Epoch no. 7 Train Loss: 0.1830 Train Accuracy: 86.7886 Validation Loss: 0.3995 Validation Accuracy: 86.5452
Epoch no. 8 Train Loss: 0.1800 Train Accuracy: 86.9009 Validation Loss: 0.4024 Validation Accuracy: 86.6450


In [ ]:
with torch.no_grad():
  total_loss_test = 0
  total_acc_test = 0
  for indx, data in enumerate(testing_dataloader):
    input, label = data
    input.to(device)
    label.to(device)

    prediction = model(input['input_ids'].squeeze(1), input['attention_mask'].squeeze(1)).squeeze(1)

    batch_loss_val = criterion(prediction, label)
    total_loss_test += batch_loss_val.item()
    acc = ((prediction).round() == label).sum().item()
    total_acc_test += acc

print(f"Accuracy Score is: {round((total_acc_test/X_test.shape[0])*100, 2)}%")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axs[0].plot(total_loss_train_plot, label='Training Loss')
axs[0].plot(total_loss_validation_plot, label='Validation Loss')
axs[0].set_title('Training and Validation Loss over Epochs')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].set_ylim([0, 1])
axs[0].legend()

axs[1].plot(total_acc_train_plot, label='Training Accuracy')
axs[1].plot(total_acc_validation_plot, label='Validation Accuracy')
axs[1].set_title('Training and Validation Accuracy over Epochs')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].set_ylim([0, 100])
axs[1].legend()

plt.tight_layout()

plt.show()